In [19]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import tiktoken
from tokenizers import Tokenizer

# Global Variable

In [37]:
block_size = 32 # Panjang Input Train
n_head = 4 # Jumlah Heads
n_embd = 64 # Dimensi Embedding
n_layer = 2 # Jumlah Block Decoder Layer
batch_size = 64 # Ukuran Batch

max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 200

dropout = 0.2
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [39]:
torch.manual_seed(6699)

# Load Dataset

In [40]:
with open('../data/LeetCodeDataset-train.txt', 'r', encoding='utf-8') as f:
    text = f.read()

## Tokenizer

### Pretrained GPT-2

In [18]:
enc = tiktoken.get_encoding("gpt2")
vocab_size = enc.n_vocab
data = torch.tensor(enc.encode(text), dtype=torch.long)

print("vocab_size", vocab_size)

vocab_size 50257


### Trained Dataset BPE

In [52]:

enc = Tokenizer.from_file("../out/bpe_tokenizer.json")
vocab_size = enc.get_vocab_size()
ids = enc.encode(text).ids
data = torch.tensor(ids, dtype=torch.long)

print("vocab_size", vocab_size)

vocab_size 10000


### Trained Dataset WordPiece

In [49]:
enc = Tokenizer.from_file("../out/wordpiece_tokenizer.json")
vocab_size = enc.get_vocab_size()
ids = enc.encode(text).ids
data = torch.tensor(ids, dtype=torch.long)

print("vocab_size", vocab_size)

vocab_size 10000


## Split

In [53]:
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# Model

## Transformer Code

In [47]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class MaskedSelfAttention(nn.Module):
    """ Satu self-attention head dengan masking untuk memastikan model tidak melihat ke token di masa depan """

    def __init__(self, head_size):
        super().__init__()
        # Layer untuk menghasilkan vektor key, query, dan value dari input
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        # Matriks lower trigangular untuk masking agar model hanya melihat token sebelumnya
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x: (batch, time, channel)
        B,T,C = x.shape
        k = self.key(x)   # Hasil key projection
        q = self.query(x) # Hasil query projection
        # Hitung attention score antar token, distandarisasi
        weight = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        # Masking agar model tidak memprediksi token masa depan
        weight = weight.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        weight = F.softmax(weight, dim=-1)
        weight = self.dropout(weight)
        # Hitung representasi berdasarkan attention score dan value
        v = self.value(x)
        out = weight @ v
        return out

class MultiHeadAttention(nn.Module):
    """ Beberapa self-attention head yang bekerja paralel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        # Initialize beberapa head self-attention
        self.heads = nn.ModuleList([MaskedSelfAttention(head_size) for _ in range(num_heads)])
        # Proyeksikan gabungan hasil seluruh head ke dimensi awal
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Concat output dari semua head
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        # Proyeksikan kembali ke dimensi semula dan beri dropout
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ Simple linear layer """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class DecoderBlock(nn.Module):
    """ Blok decoder Transformer: proses komunikasi (attention) dan komputasi (feed-forward) """

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        # Self-attention dengan banyak head
        self.sa = MultiHeadAttention(n_head, head_size)
        # Feed forward setelah attention
        self.ffwd = FeedFoward(n_embd)
        # Normalisasi layer sebelum proses utama
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # Tambahkan residual connection dan normalisasi
        # Pre-LN
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x
    
class TransformerDecoderOnly(nn.Module):
    """  Model Transformer Decoder-only  """

    def __init__(self):
        super().__init__()
        # Embedding untuk token dan posisi
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        # Sekuensial blok Transformer decoder
        self.blocks = nn.Sequential(*[DecoderBlock(n_embd, n_head=n_head) for _ in range(n_layer)])
        # Normalisasi terakhir
        self.ln_f = nn.LayerNorm(n_embd)
        # Proyeksi ke ukuran kosakata untuk prediksi token berikutnya
        self.lm_head = nn.Linear(n_embd, vocab_size)
        # Inisialisasi bobot
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # Buat embedding token dan posisi
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        # Masukkan ke blok transformer
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        # Hitung loss jika ada target
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        """Menghasilkan urutan token baru berdasarkan konteks awal `idx`"""
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :] # hanya waktu terakhir
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

## Training

In [54]:
model = TransformerDecoderOnly()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


1.39176 M parameters
step 0: train loss 9.2149, val loss 9.2130
step 500: train loss 6.4064, val loss 6.4060
step 1000: train loss 5.4754, val loss 5.5615
step 1500: train loss 4.9342, val loss 5.0924
step 2000: train loss 4.5858, val loss 4.7872
step 2500: train loss 4.3061, val loss 4.5902
step 3000: train loss 4.1113, val loss 4.4621
step 3500: train loss 3.9536, val loss 4.3574
step 4000: train loss 3.8502, val loss 4.2827
step 4500: train loss 3.7242, val loss 4.2594
step 4999: train loss 3.6402, val loss 4.1811


# Experiment

## Test Case

In [62]:
tc_1_q = """You are given a string target, an array of strings words, and an integer array costs, both arrays of the same length.
Imagine an empty string s.
You can perform the following operation any number of times (including zero):

Choose an index i in the range [0, words.length - 1].
Append words[i] to s.
The cost of operation is costs[i].

Return the minimum cost to make s equal to target. If it's not possible, return -1.
 
Example 1:

Input: target = "abcdef", words = ["abdef","abc","d","def","ef"], costs = [100,1,1,10,5]
Output: 7
Explanation:
The minimum cost can be achieved by performing the following operations:

Select index 1 and append "abc" to s at a cost of 1, resulting in s = "abc".
Select index 2 and append "d" to s at a cost of 1, resulting in s = "abcd".
Select index 4 and append "ef" to s at a cost of 5, resulting in s = "abcdef".


Example 2:

Input: target = "aaaa", words = ["z","zz","zzz"], costs = [1,10,100]
Output: -1
Explanation:
It is impossible to make s equal to target, so we return -1.

 
Constraints:

1 <= target.length <= 2000
1 <= words.length == costs.length <= 50
1 <= words[i].length <= target.length
target and words[i] consist only of lowercase English letters.
1 <= costs[i] <= 105"""

tc_1_a = """```python
from typing import List
from functools import lru_cache

class Solution:
    def minimumCost(self, target: str, words: List[str], costs: List[int]) -> int:
        
        @lru_cache(None)
        def dp(i):
            if i == 0:
                return 0
            
            min_cost = float('inf')
            for j, word in enumerate(words):
                if i >= len(word) and target[i-len(word):i] == word:
                    min_cost = min(min_cost, dp(i - len(word)) + costs[j])
            
            return min_cost
        
        result = dp(len(target))
        return result if result != float('inf') else -1
```

### Explanation:
- We use a recursive function `dp(i)` to find the minimum cost to form the substring `target[0:i]`.
- `dp(0)` is `0` because forming an empty string has no cost.
- For each position `i` in the target string, we check if any word in the `words` list can be used to extend a previous substring to reach `i`.
- If `target[i-len(word):i]` matches a word, we consider the cost of forming `target[0:i-len(word)]` plus the cost of appending that word.
- We use `lru_cache` to memoize results and avoid redundant calculations.
- Finally, we check the result for `dp(len(target))`. If it's still `float('inf')`, it means it's not possible to form the target string, so we return `-1`. Otherwise, we return the computed minimum cost.
"""

tc_2_q = """You are given an array of integers nums of length n and a positive integer k.
The power of an array is defined as:

Its maximum element if all of its elements are consecutive and sorted in ascending order.
-1 otherwise.

You need to find the power of all subarrays of nums of size k.
Return an integer array results of size n - k + 1, where results[i] is the power of nums[i..(i + k - 1)].
 
Example 1:

Input: nums = [1,2,3,4,3,2,5], k = 3
Output: [3,4,-1,-1,-1]
Explanation:
There are 5 subarrays of nums of size 3:

[1, 2, 3] with the maximum element 3.
[2, 3, 4] with the maximum element 4.
[3, 4, 3] whose elements are not consecutive.
[4, 3, 2] whose elements are not sorted.
[3, 2, 5] whose elements are not consecutive.


Example 2:

Input: nums = [2,2,2,2,2], k = 4
Output: [-1,-1]

Example 3:

Input: nums = [3,2,3,2,3,2], k = 2
Output: [-1,3,-1,3,-1]

 
Constraints:

1 <= n == nums.length <= 500
1 <= nums[i] <= 105
1 <= k <= n"""

tc_2_a = """```python
from typing import List

class Solution:
    def resultsArray(self, nums: List[int], k: int) -> List[int]:
        def is_consecutive_sorted(subarray):
            return subarray == list(range(min(subarray), max(subarray) + 1))
        
        results = []
        for i in range(len(nums) - k + 1):
            subarray = nums[i:i + k]
            if is_consecutive_sorted(subarray):
                results.append(max(subarray))
            else:
                results.append(-1)
        
        return results
```

This solution defines a helper function `is_consecutive_sorted` to check if a subarray is consecutive and sorted in ascending order. It then iterates over all possible subarrays of length `k` in the input array `nums`, checks if each subarray meets the criteria, and appends the appropriate value (either the maximum element or -1) to the results list.
"""

In [65]:
tc_1_q_tokens = enc.encode(tc_1_q).ids
tc_1_a_tokens = enc.encode(tc_1_a).ids
tc_2_q_tokens = enc.encode(tc_2_q).ids
tc_2_a_tokens = enc.encode(tc_2_a).ids

In [91]:
context = torch.tensor([tc_1_q_tokens], dtype=torch.long, device=device)
tc_1_r_tokens = m.generate(context, max_new_tokens=len(tc_1_a_tokens))[0].tolist()
answer_only = tc_1_r_tokens[len(tc_1_q_tokens):]

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu




In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score




def get_metrics(y_pred, y_true):
    print("loss:", loss.item())
    print("precision:", precision_score(y_true, y_pred, average='micro'))
    print("recall:", recall_score(y_true, y_pred, average='micro'))
    print("f1-score:", f1_score(y_true, y_pred, average='micro'))

In [76]:
print(len(tc_1_r_tokens))
print(len(tc_1_a_tokens))

251
251


In [77]:
print(token_level_metrics(tc_1_r_tokens, tc_1_a_tokens))

{'precision': 0.01195219123505976, 'recall': 0.01195219123505976, 'f1': 0.01195219123505976}


## Result

### Char encoding

In [15]:
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

	 F ageate elements the current areal bet low"subdy "lo"4"].

 
Evalting = nums of not in revertirdia nodes in the next orde.
Return tis = "
Output: [[[:1]::-1]
Output: [10]]


 
Constraints:

1 <= nums2.length <= 31
1 orr negativeNode.valuates the highouse treehe lowisth linke half. The of midd O() k if \(nums) corner` function - beca move alie, co is no.
           # Defs the shigne than or ite bisect is slowive using th subs ode binay lary prefind to size ways a wercause the ch bit of linked n


In [8]:
# Your input string
input_str = '''You are given an integer n and a 2D integer array queries.
There are n cities numbered from 0 to n - 1. Initially, there is a unidirectional road from city i to city i + 1 for all 0 <= i < n - 1.
queries[i] = [ui, vi] represents the addition of a new unidirectional road from city ui to city vi. After each query, you need to find the length of the shortest path from city 0 to city n - 1.
Return an array answer where for each i in the range [0, queries.length - 1], answer[i] is the length of the shortest path from city 0 to city n - 1 after processing the first i + 1 queries.
 
Example 1:

Input: n = 5, queries = [[2,4],[0,2],[0,4]]
Output: [3,2,1]
Explanation: 

After the addition of the road from 2 to 4, the length of the shortest path from 0 to 4 is 3.

After the addition of the road from 0 to 2, the length of the shortest path from 0 to 4 is 2.

After the addition of the road from 0 to 4, the length of the shortest path from 0 to 4 is 1.

Example 2:

Input: n = 4, queries = [[0,3],[0,2]]
Output: [1,1]
Explanation:

After the addition of the road from 0 to 3, the length of the shortest path from 0 to 3 is 1.

After the addition of the road from 0 to 2, the length of the shortest path remains 1.

 
Constraints:

3 <= n <= 500
1 <= queries.length <= 500
queries[i].length == 2
0 <= queries[i][0] < queries[i][1] < n
1 < queries[i][1] - queries[i][0]
There are no repeated roads among the queries.
'''

# Encode to token IDs
context_tokens = encode(input_str)  # this returns a list of ints

# Convert to tensor, shape (1, seq_len)
context = torch.tensor([context_tokens], dtype=torch.long, device=device)

print(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))

You are given an integer n and a 2D integer array queries.
There are n cities numbered from 0 to n - 1. Initially, there is a unidirectional road from city i to city i + 1 for all 0 <= i < n - 1.
queries[i] = [ui, vi] represents the addition of a new unidirectional road from city ui to city vi. After each query, you need to find the length of the shortest path from city 0 to city n - 1.
Return an array answer where for each i in the range [0, queries.length - 1], answer[i] is the length of the shortest path from city 0 to city n - 1 after processing the first i + 1 queries.
 
Example 1:

Input: n = 5, queries = [[2,4],[0,2],[0,4]]
Output: [3,2,1]
Explanation: 

After the addition of the road from 2 to 4, the length of the shortest path from 0 to 4 is 3.

After the addition of the road from 0 to 2, the length of the shortest path from 0 to 4 is 2.

After the addition of the road from 0 to 4, the length of the shortest path from 0 to 4 is 1.

Example 2:

Input: n = 4, queries = [[0,3],[0

### Tiktoken GPT-2

In [ ]:
# 7 menit training

print(enc.decode(m.generate(context, max_new_tokens=500)[0].tolist()))

! distinct answers = rain water (the wall name is incremented.

The root-to-leaf path in the dictionary must be validated where it's rows = 2, labeled from nums 0 to updates the prefix = [[1,1, 0],[0]))
        max_product = max(sequence)
        word_to_binary_sum = 0
        
          # Add all ranges, and max_index that cover all empty characters are within the queue
        return len(rat", nums of each unique letters and a positions where:
             columnNumber, 0: (representedps[i, j)) if grid[0][j] == 0:
               # A string return -1
         return dp[x][y]
```

This solution uses a two-pointer technique to perform binary search once. The `is_seen`next` to count the bucket sort to flatten the binary search approach by a heap that reflects the BFS is constructed from a repeating and on the following follows:

The overall process is edge cases where the width counter ( iterate through the list of the input list `k` and `[i]` such that `1` and the number of unique paths

### Trained BPE

#### 2.9 M Params

In [36]:
# Your input string
input_str = '''You are given an integer n and a 2D integer array queries.
There are n cities numbered from 0 to n - 1. Initially, there is a unidirectional road from city i to city i + 1 for all 0 <= i < n - 1.
queries[i] = [ui, vi] represents the addition of a new unidirectional road from city ui to city vi. After each query, you need to find the length of the shortest path from city 0 to city n - 1.
Return an array answer where for each i in the range [0, queries.length - 1], answer[i] is the length of the shortest path from city 0 to city n - 1 after processing the first i + 1 queries.
 
Example 1:

Input: n = 5, queries = [[2,4],[0,2],[0,4]]
Output: [3,2,1]
Explanation: 

After the addition of the road from 2 to 4, the length of the shortest path from 0 to 4 is 3.

After the addition of the road from 0 to 2, the length of the shortest path from 0 to 4 is 2.

After the addition of the road from 0 to 4, the length of the shortest path from 0 to 4 is 1.

Example 2:

Input: n = 4, queries = [[0,3],[0,2]]
Output: [1,1]
Explanation:

After the addition of the road from 0 to 3, the length of the shortest path from 0 to 3 is 1.

After the addition of the road from 0 to 2, the length of the shortest path remains 1.

 
Constraints:

3 <= n <= 500
1 <= queries.length <= 500
queries[i].length == 2
0 <= queries[i][0] < queries[i][1] < n
1 < queries[i][1] - queries[i][0]
There are no repeated roads among the queries.
'''

# Encode to token IDs
context_tokens = enc.encode(input_str)  # this returns a list of ints
context_tokens = context_tokens.ids

# Convert to tensor, shape (1, seq_len)
context = torch.tensor([context_tokens], dtype=torch.long, device=device)

print(enc.decode(m.generate(context, max_new_tokens=1000)[0].tolist()))

You are given an integer  n and  a 2D integer array  queries . 
 There are n  cities  numbered from 0 to n -  1.  Initially,  there is a  uni directional  road  from  city  i to  city  i +  1  for all  0 <= i <  n -  1. 
 queries[i] = [ ui, vi ] represents the  addition  of a  new  uni directional  road  from  city  u i to  city  vi . After  each  query,  you need to  find the  length of the  shortest path  from  city  0 to  city  n -  1. 
 Return an array answer  where  for each  i  in the range [0,  queries .length  -  1],  answer[i] is the  length of the  shortest path  from  city  0 to  city  n - 1  after  proces sing the  first  i +  1  queries . 
   
 Example 1: 
 
 Input: n = 5,  queries = [ [2, 4],[ 0, 2],[0, 4] ] 
 Output: [ 3,2, 1] 
 Explanation:  
 
 After the  addition  of the  road  from  2 to  4 , the  length of the  shortest path  from 0 to  4 is  3. 
 
 After the  addition  of the  road  from 0 to  2, the  length of the  shortest path  from 0 to  4 is  2. 
 
 After the 

### 1.3 M Params

In [55]:
# Your input string
input_str = '''You are given an integer n and a 2D integer array queries.
There are n cities numbered from 0 to n - 1. Initially, there is a unidirectional road from city i to city i + 1 for all 0 <= i < n - 1.
queries[i] = [ui, vi] represents the addition of a new unidirectional road from city ui to city vi. After each query, you need to find the length of the shortest path from city 0 to city n - 1.
Return an array answer where for each i in the range [0, queries.length - 1], answer[i] is the length of the shortest path from city 0 to city n - 1 after processing the first i + 1 queries.
 
Example 1:

Input: n = 5, queries = [[2,4],[0,2],[0,4]]
Output: [3,2,1]
Explanation: 

After the addition of the road from 2 to 4, the length of the shortest path from 0 to 4 is 3.

After the addition of the road from 0 to 2, the length of the shortest path from 0 to 4 is 2.

After the addition of the road from 0 to 4, the length of the shortest path from 0 to 4 is 1.

Example 2:

Input: n = 4, queries = [[0,3],[0,2]]
Output: [1,1]
Explanation:

After the addition of the road from 0 to 3, the length of the shortest path from 0 to 3 is 1.

After the addition of the road from 0 to 2, the length of the shortest path remains 1.

 
Constraints:

3 <= n <= 500
1 <= queries.length <= 500
queries[i].length == 2
0 <= queries[i][0] < queries[i][1] < n
1 < queries[i][1] - queries[i][0]
There are no repeated roads among the queries.
'''

# Encode to token IDs
context_tokens = enc.encode(input_str)  # this returns a list of ints
context_tokens = context_tokens.ids

# Convert to tensor, shape (1, seq_len)
context = torch.tensor([context_tokens], dtype=torch.long, device=device)

print(enc.decode(m.generate(context, max_new_tokens=1000)[0].tolist()))

You are given an integer  n and  a 2D integer array  queries . 
 There are n  cities  numbered from 0 to n -  1.  Initially,  there is a  uni directional  road  from  city  i to  city  i +  1  for all  0 <= i <  n -  1. 
 queries[i] = [ ui, vi ] represents the  addition  of a  new  uni directional  road  from  city  u i to  city  vi . After  each  query,  you need to  find the  length of the  shortest path  from  city  0 to  city  n -  1. 
 Return an array answer  where  for each  i  in the range [0,  queries .length  -  1],  answer[i] is the  length of the  shortest path  from  city  0 to  city  n - 1  after  proces sing the  first  i +  1  queries . 
   
 Example 1: 
 
 Input: n = 5,  queries = [ [2, 4],[ 0, 2],[0, 4] ] 
 Output: [ 3,2, 1] 
 Explanation:  
 
 After the  addition  of the  road  from  2 to  4 , the  length of the  shortest path  from 0 to  4 is  3. 
 
 After the  addition  of the  road  from 0 to  2, the  length of the  shortest path  from 0 to  4 is  2. 
 
 After the 

# Tetot